In [ ]:
!pip install vllm unidecode langdetect pandas tqdm

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
import json
import os
import random
import string
import re
from pathlib import Path
from tqdm import tqdm

from unidecode import unidecode
from vllm import LLM, SamplingParams
from huggingface_hub import snapshot_download
from transformers import AutoTokenizer
from langdetect import detect, detect_langs
from langdetect.lang_detect_exception import LangDetectException

# ------------ CONFIG ------------
MODELO = "google/gemma-3-4b-it"   # repo HF o ruta local meta-llama/Llama-3.1-8B-Instruct TinyLlama/TinyLlama-1.1B-Chat-v1.0 Qwen/Qwen3-4B-Instruct-2507
CANTIDAD = 20000
MAX_PREGUNTAS_FILTRAR = 20000  # Limitar el número de preguntas a procesar para filtrar
BENCHMARK_FILE = "gonza_qa.json"
OUTPUT_FILE = f"gonza_evaluacion_real_{MODELO.split('/')[-1]}.json"
MODELS_DIR = Path("./models")                   # dónde guardar los modelos
os.environ["HUGGINGFACE_HUB_TOKEN"] = ""
# ---------------------------------

print(f"Configuración:")
print(f"- Modelo: {MODELO}")
print(f"- Cantidad de preguntas para evaluación: {CANTIDAD}")
print(f"- Máximo de preguntas a procesar para filtrar: {MAX_PREGUNTAS_FILTRAR}")
print(f"- Archivo benchmark: {BENCHMARK_FILE}")
print("-" * 50)

# Ejemplos few-shot específicos de Latinoamérica
FEW_SHOT_EXAMPLES = [
    {
        "pregunta": "¿Cuál es la capital de Argentina?",
        "respuesta": "Buenos Aires"
    },
    {
        "pregunta": "¿En qué año se independizó México?",
        "respuesta": "1810"
    },
    {
        "pregunta": "¿Cuál es la moneda oficial de Chile?",
        "respuesta": "Peso chileno"
    },
    {
        "pregunta": "¿Quién fue el libertador de Venezuela?",
        "respuesta": "Simón Bolívar"
    },
    {
        "pregunta": "¿Cuál es el río más largo de Sudamérica?",
        "respuesta": "Río Amazonas"
    },
    {
        "pregunta": "¿En qué país se encuentra Machu Picchu?",
        "respuesta": "Perú"
    },
    {
        "pregunta": "¿Cuál es la capital de Colombia?",
        "respuesta": "Bogotá"
    },
    {
        "pregunta": "¿Qué escritor colombiano ganó el Premio Nobel de Literatura?",
        "respuesta": "Gabriel García Márquez"
    },
    {
        "pregunta": "¿Cuál es el país más grande de América del Sur?",
        "respuesta": "Brasil"
    },
    {
        "pregunta": "¿En qué océano se encuentran las Islas Galápagos?",
        "respuesta": "Océano Pacífico"
    },
    {
        "pregunta": "¿Cuál es la capital de Ecuador?",
        "respuesta": "Quito"
    },
    {
        "pregunta": "¿Qué cordillera atraviesa América del Sur?",
        "respuesta": "Cordillera de los Andes"
    },
    {
        "pregunta": "¿Cuál es la moneda oficial de Brasil?",
        "respuesta": "Real brasileño"
    },
    {
        "pregunta": "¿En qué país nació el tango?",
        "respuesta": "Argentina"
    },
    {
        "pregunta": "¿Cuál es la capital de Uruguay?",
        "respuesta": "Montevideo"
    },
    {
        "pregunta": "¿Qué país tiene forma de bota en América del Sur?",
        "respuesta": "Chile"
    },
    {
        "pregunta": "¿Cuál es el desierto más árido del mundo ubicado en Chile?",
        "respuesta": "Desierto de Atacama"
    },
    {
        "pregunta": "¿En qué país se encuentra el Salar de Uyuni?",
        "respuesta": "Bolivia"
    },
    {
        "pregunta": "¿Cuál es la capital de Paraguay?",
        "respuesta": "Asunción"
    },
    {
        "pregunta": "¿Qué idioma se habla en Brasil?",
        "respuesta": "Portugués"
    }
]

def is_local_path(p: str) -> bool:
    return Path(p).expanduser().exists()

def sanitize(name: str) -> str:
    return "".join(ch if ch.isalnum() or ch in "._-+=" else "_" for ch in name)

def ensure_local_model(model_id_or_path: str, local_dir: Path) -> str:
    if is_local_path(model_id_or_path):
        return str(Path(model_id_or_path).expanduser().resolve())
    
    local_dir.mkdir(parents=True, exist_ok=True)
    dst = local_dir / sanitize(model_id_or_path)
    
    # Bypass específico para google/gemma-3-4b-it
    if "google/gemma" in model_id_or_path.lower():
        print(f"Aplicando bypass para modelo Gemma: {model_id_or_path}")
        try:
            # Intentar descarga con patrones más amplios para Gemma
            snapshot_download(
                repo_id=model_id_or_path,
                local_dir=str(dst),
                local_dir_use_symlinks=False,
                # Patrones más amplios para Gemma
                allow_patterns=[
                    "*.json", "*.txt", "*.model", "*.safetensors", "*.bin",
                    "tokenizer*", "config*", "generation_config*",
                    "model*", "pytorch_model*", "special_tokens_map*"
                ],
            )
        except Exception as e1:
            print(f"Primer intento falló: {e1}")
            try:
                # Segundo intento: descarga completa sin filtros
                print("Intentando descarga completa sin filtros...")
                snapshot_download(
                    repo_id=model_id_or_path,
                    local_dir=str(dst),
                    local_dir_use_symlinks=False,
                )
            except Exception as e2:
                print(f"Segundo intento falló: {e2}")
                # Tercer intento: usar directamente el repo_id para vLLM
                print("Usando repo_id directamente para vLLM...")
                return model_id_or_path
    else:
        # Para otros modelos, usar el método original
        snapshot_download(
            repo_id=model_id_or_path,
            local_dir=str(dst),
            local_dir_use_symlinks=False,
            allow_patterns=[
                "config.json", "generation_config.json",
                "tokenizer.json", "tokenizer.model", "tokenizer_config.json",
                "special_tokens_map.json", "merges.txt", "vocab.json",
                "*.safetensors", "model.safetensors.index.json", "pytorch_model.bin",
            ],
        )
    
    return str(dst)

def is_spanish_text(text: str) -> bool:
    """Detecta si el texto está en español"""
    if not text or len(text.strip()) < 3:
        return False
    
    try:
        # Usar detect_langs para obtener probabilidades
        languages = detect_langs(text)
        
        # Buscar español en los resultados
        for lang in languages:
            if lang.lang == 'es' and lang.prob > 0.55:
                return True
        
        # Si no hay alta confianza, usar detect simple
        detected_lang = detect(text)
        return detected_lang == 'es'
        
    except:
        # Si langdetect falla completamente, considerar como no español
        return False

def filter_spanish_questions_sequential(benchmark_data, max_items=None):
    """Filtra preguntas que estén completamente en español usando procesamiento secuencial"""
    filtered_data = []
    filtered_count = 0
    
    # Limitar el número de items a procesar si se especifica
    if max_items and max_items < len(benchmark_data):
        print(f"Limitando procesamiento a {max_items} de {len(benchmark_data)} preguntas totales")
        data_to_process = benchmark_data[:max_items]
    else:
        data_to_process = benchmark_data
    
    print(f"Filtrando {len(data_to_process)} preguntas...")
    
    for item in tqdm(data_to_process, desc="Filtrando preguntas"):
        pregunta = item.get('pregunta', '')
        respuesta = item.get('respuesta_correcta', '')
        
        # Verificar que tanto pregunta como respuesta estén en español
        if is_spanish_text(pregunta) and is_spanish_text(respuesta):
            filtered_data.append(item)
        else:
            filtered_count += 1
    
    print(f"Filtradas {filtered_count} preguntas (no español)")
    print(f"Preguntas válidas en español: {len(filtered_data)}")
    return filtered_data

def normalizar_texto(texto):
    if not isinstance(texto, str):
        return ""
    texto = texto.lower()
    texto = unidecode(texto)
    texto = texto.translate(str.maketrans('', '', string.punctuation))
    return texto.strip()

# --- Carga benchmark ---
if not os.path.exists(BENCHMARK_FILE):
    print(f"Error: No se encuentra '{BENCHMARK_FILE}'. Ejecuta el script 2 primero.")
    raise SystemExit(1)

with open(BENCHMARK_FILE, 'r', encoding='utf-8') as f:
    benchmark_data = json.load(f)

if len(benchmark_data) == 0:
    print("Error: El benchmark está vacío.")
    raise SystemExit(1)

# Filtrar preguntas en español usando procesamiento secuencial
print(f"Datos originales: {len(benchmark_data)} preguntas")
# benchmark_data_spanish = filter_spanish_questions_sequential(benchmark_data, MAX_PREGUNTAS_FILTRAR)
benchmark_data_spanish = benchmark_data
print(f"Después del filtro de español: {len(benchmark_data_spanish)} preguntas")

if len(benchmark_data_spanish) == 0:
    print("Error: No hay preguntas válidas en español después del filtro.")
    raise SystemExit(1)

# Verificar si tenemos suficientes preguntas para la muestra
if len(benchmark_data_spanish) < CANTIDAD:
    print(f"Advertencia: Solo hay {len(benchmark_data_spanish)} preguntas válidas, usando todas para la evaluación.")
    CANTIDAD = len(benchmark_data_spanish)

benchmark_sample = random.sample(benchmark_data_spanish, min(len(benchmark_data_spanish), CANTIDAD))
print(f"Muestra seleccionada: {len(benchmark_sample)} preguntas")

# --- Modelo local (descarga si hace falta) ---
modelo_local = ensure_local_model(MODELO, MODELS_DIR)
print(f"Modelo preparado en: {modelo_local}")

# --- Tokenizer (para chat template si existe) ---
# Bypass para tokenizer de Gemma
if "google/gemma" in MODELO.lower() and modelo_local == MODELO:
    # Si no se pudo descargar localmente, usar directamente el repo
    print("Cargando tokenizer directamente desde Hugging Face...")
    tok = AutoTokenizer.from_pretrained(
        MODELO,
        trust_remote_code=True
    )
else:
    tok = AutoTokenizer.from_pretrained(
        modelo_local,
        trust_remote_code=True,
        local_files_only=True
    )

def create_few_shot_prompt(pregunta: str) -> str:
    """Crea un prompt con ejemplos few-shot de Latinoamérica"""
    system_message = """Eres un asistente experto en conocimientos de Latinoamérica. Responde de manera precisa y concisa en español.

Ejemplos de preguntas y respuestas sobre Latinoamérica:"""
    
    # Agregar ejemplos few-shot
    examples_text = ""
    for example in FEW_SHOT_EXAMPLES:
        examples_text += f"\nPregunta: {example['pregunta']}\nRespuesta: {example['respuesta']}\n"
    
    user_message = f"\nAhora responde esta pregunta:\nPregunta: {pregunta}\nRespuesta:"
    
    return system_message + examples_text + user_message

def format_prompt(pregunta: str) -> str:
    few_shot_content = create_few_shot_prompt(pregunta)
    
    if hasattr(tok, "apply_chat_template") and (getattr(tok, "chat_template", None) is not None):
        messages = [
            {"role": "system", "content": "Eres un asistente experto en conocimientos de Latinoamérica."},
            {"role": "user", "content": few_shot_content}
        ]
        return tok.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
    return f"<|user|>\n{few_shot_content}<|assistant|>"

prompts = [format_prompt(item['pregunta']) for item in benchmark_sample]
respuestas_correctas_sample = [item['respuesta_correcta'] for item in benchmark_sample]

print(f"\nCargando modelo '{MODELO}' en la GPU...")
sampling_params = SamplingParams(temperature=0.1, top_p=0.9, max_tokens=128)

# Configuración específica para Gemma
if "google/gemma" in MODELO.lower():
    print("Configurando vLLM para modelo Gemma...")
    try:
        llm = LLM(
            model=modelo_local, 
            gpu_memory_utilization=0.7, 
            dtype="float16", 
            trust_remote_code=True,
            # Configuraciones adicionales para Gemma
            max_model_len=4096,
            enforce_eager=True
        )
    except Exception as e:
        print(f"Error con configuración específica de Gemma: {e}")
        print("Intentando con configuración estándar...")
        llm = LLM(model=modelo_local, gpu_memory_utilization=0.7, dtype="float16", trust_remote_code=True)
else:
    llm = LLM(model=modelo_local, gpu_memory_utilization=0.7, dtype="float16", trust_remote_code=True)

print("¡Modelo cargado! Iniciando generación de respuestas...")
outputs = llm.generate(prompts, sampling_params)
print("¡Generación completa! Evaluando respuestas...")

total_preguntas = 0
correctas = 0
resultados_evaluacion = []

for i, output in enumerate(outputs):
    pregunta_original = benchmark_sample[i]['pregunta']
    respuesta_llm = output.outputs[0].text.strip()
    respuesta_correcta_gt = respuestas_correctas_sample[i]

    # Verificar que la respuesta del LLM esté en español
    respuesta_en_espanol = respuesta_llm
    
    norm_correcta = normalizar_texto(respuesta_correcta_gt)
    norm_llm = normalizar_texto(respuesta_llm)
    es_correcta = (norm_correcta in norm_llm) and (norm_correcta != "") and respuesta_en_espanol

    if es_correcta:
        correctas += 1
    total_preguntas += 1

    evaluacion = "CORRECTO" if es_correcta else "INCORRECTO"
    if not respuesta_en_espanol:
        evaluacion += " (No español)"

    resultados_evaluacion.append({
        "pregunta": pregunta_original,
        "respuesta_correcta": respuesta_correcta_gt,
        "respuesta_llm": respuesta_llm,
        "evaluacion": evaluacion,
        "respuesta_en_espanol": respuesta_en_espanol
    })

print("\n--- Tabla de Resultados de Evaluación (Muestra de 10) ---")
for res in resultados_evaluacion[:10]:
    print(f"  P: {res['pregunta']}")
    print(f"  R. Correcta: {res['respuesta_correcta']}")
    print(f"  R. LLM: {res['respuesta_llm']} -> {res['evaluacion']}")
    print("  ---")

if total_preguntas > 0:
    puntaje = (correctas / total_preguntas) * 100
    respuestas_espanol = sum(1 for r in resultados_evaluacion if r['respuesta_en_espanol'])
    print(f"\n Resultado Final ({MODELO}): {correctas} de {total_preguntas} correctas ({puntaje:.1f}%)")
    print(f" Respuestas en español: {respuestas_espanol} de {total_preguntas} ({(respuestas_espanol/total_preguntas)*100:.1f}%)")
else:
    print("No se evaluaron preguntas.")

with open(OUTPUT_FILE, 'w', encoding='utf-8') as f:
    json.dump(resultados_evaluacion, f, ensure_ascii=False, indent=4)

print(f"Resultados detallados guardados en '{OUTPUT_FILE}'")
print("-------------------------------------------------")

In [ ]:
import json
import os
import random
import string
import re

from pathlib import Path
from tqdm import tqdm

from unidecode import unidecode
from vllm import LLM, SamplingParams
from huggingface_hub import snapshot_download
from transformers import AutoTokenizer
from langdetect import detect, detect_langs
from langdetect.lang_detect_exception import LangDetectException

# ------------ CONFIG ------------
MODELO = "google/gemma-3-4b-it"   # repo HF o ruta local meta-llama/Llama-3.1-8B-Instruct TinyLlama/TinyLlama-1.1B-Chat-v1.0 Qwen/Qwen3-4B-Instruct-2507
CANTIDAD = 20000
MAX_PREGUNTAS_FILTRAR = 20000  # Limitar el número de preguntas a procesar para filtrar
BENCHMARK_FILE = "usa_qa.json"
OUTPUT_FILE = f"usa_evaluacion_real_{MODELO.split('/')[-1]}.json"
MODELS_DIR = Path("./models")                   # dónde guardar los modelos
os.environ["HUGGINGFACE_HUB_TOKEN"] = ""
# ---------------------------------

print(f"Configuración:")
print(f"- Modelo: {MODELO}")
print(f"- Cantidad de preguntas para evaluación: {CANTIDAD}")
print(f"- Máximo de preguntas a procesar para filtrar: {MAX_PREGUNTAS_FILTRAR}")
print(f"- Archivo benchmark: {BENCHMARK_FILE}")
print("-" * 50)

# Ejemplos few-shot específicos de Latinoamérica
FEW_SHOT_EXAMPLES = [
    {
        "pregunta": "¿Cuál es la capital de Argentina?",
        "respuesta": "Buenos Aires"
    },
    {
        "pregunta": "¿En qué año se independizó México?",
        "respuesta": "1810"
    },
    {
        "pregunta": "¿Cuál es la moneda oficial de Chile?",
        "respuesta": "Peso chileno"
    },
    {
        "pregunta": "¿Quién fue el libertador de Venezuela?",
        "respuesta": "Simón Bolívar"
    },
    {
        "pregunta": "¿Cuál es el río más largo de Sudamérica?",
        "respuesta": "Río Amazonas"
    },
    {
        "pregunta": "¿En qué país se encuentra Machu Picchu?",
        "respuesta": "Perú"
    },
    {
        "pregunta": "¿Cuál es la capital de Colombia?",
        "respuesta": "Bogotá"
    },
    {
        "pregunta": "¿Qué escritor colombiano ganó el Premio Nobel de Literatura?",
        "respuesta": "Gabriel García Márquez"
    },
    {
        "pregunta": "¿Cuál es el país más grande de América del Sur?",
        "respuesta": "Brasil"
    },
    {
        "pregunta": "¿En qué océano se encuentran las Islas Galápagos?",
        "respuesta": "Océano Pacífico"
    },
    {
        "pregunta": "¿Cuál es la capital de Ecuador?",
        "respuesta": "Quito"
    },
    {
        "pregunta": "¿Qué cordillera atraviesa América del Sur?",
        "respuesta": "Cordillera de los Andes"
    },
    {
        "pregunta": "¿Cuál es la moneda oficial de Brasil?",
        "respuesta": "Real brasileño"
    },
    {
        "pregunta": "¿En qué país nació el tango?",
        "respuesta": "Argentina"
    },
    {
        "pregunta": "¿Cuál es la capital de Uruguay?",
        "respuesta": "Montevideo"
    },
    {
        "pregunta": "¿Qué país tiene forma de bota en América del Sur?",
        "respuesta": "Chile"
    },
    {
        "pregunta": "¿Cuál es el desierto más árido del mundo ubicado en Chile?",
        "respuesta": "Desierto de Atacama"
    },
    {
        "pregunta": "¿En qué país se encuentra el Salar de Uyuni?",
        "respuesta": "Bolivia"
    },
    {
        "pregunta": "¿Cuál es la capital de Paraguay?",
        "respuesta": "Asunción"
    },
    {
        "pregunta": "¿Qué idioma se habla en Brasil?",
        "respuesta": "Portugués"
    }
]

def is_local_path(p: str) -> bool:
    return Path(p).expanduser().exists()

def sanitize(name: str) -> str:
    return "".join(ch if ch.isalnum() or ch in "._-+=" else "_" for ch in name)

def ensure_local_model(model_id_or_path: str, local_dir: Path) -> str:
    if is_local_path(model_id_or_path):
        return str(Path(model_id_or_path).expanduser().resolve())
    
    local_dir.mkdir(parents=True, exist_ok=True)
    dst = local_dir / sanitize(model_id_or_path)
    
    # Bypass específico para google/gemma-3-4b-it
    if "google/gemma" in model_id_or_path.lower():
        print(f"Aplicando bypass para modelo Gemma: {model_id_or_path}")
        try:
            # Intentar descarga con patrones más amplios para Gemma
            snapshot_download(
                repo_id=model_id_or_path,
                local_dir=str(dst),
                local_dir_use_symlinks=False,
                # Patrones más amplios para Gemma
                allow_patterns=[
                    "*.json", "*.txt", "*.model", "*.safetensors", "*.bin",
                    "tokenizer*", "config*", "generation_config*",
                    "model*", "pytorch_model*", "special_tokens_map*"
                ],
            )
        except Exception as e1:
            print(f"Primer intento falló: {e1}")
            try:
                # Segundo intento: descarga completa sin filtros
                print("Intentando descarga completa sin filtros...")
                snapshot_download(
                    repo_id=model_id_or_path,
                    local_dir=str(dst),
                    local_dir_use_symlinks=False,
                )
            except Exception as e2:
                print(f"Segundo intento falló: {e2}")
                # Tercer intento: usar directamente el repo_id para vLLM
                print("Usando repo_id directamente para vLLM...")
                return model_id_or_path
    else:
        # Para otros modelos, usar el método original
        snapshot_download(
            repo_id=model_id_or_path,
            local_dir=str(dst),
            local_dir_use_symlinks=False,
            allow_patterns=[
                "config.json", "generation_config.json",
                "tokenizer.json", "tokenizer.model", "tokenizer_config.json",
                "special_tokens_map.json", "merges.txt", "vocab.json",
                "*.safetensors", "model.safetensors.index.json", "pytorch_model.bin",
            ],
        )
    
    return str(dst)

def is_spanish_text(text: str) -> bool:
    """Detecta si el texto está en español"""
    if not text or len(text.strip()) < 3:
        return False
    
    try:
        # Usar detect_langs para obtener probabilidades
        languages = detect_langs(text)
        
        # Buscar español en los resultados
        for lang in languages:
            if lang.lang == 'es' and lang.prob > 0.55:
                return True
        
        # Si no hay alta confianza, usar detect simple
        detected_lang = detect(text)
        return detected_lang == 'es'
        
    except:
        # Si langdetect falla completamente, considerar como no español
        return False

def filter_spanish_questions_sequential(benchmark_data, max_items=None):
    """Filtra preguntas que estén completamente en español usando procesamiento secuencial"""
    filtered_data = []
    filtered_count = 0
    
    # Limitar el número de items a procesar si se especifica
    if max_items and max_items < len(benchmark_data):
        print(f"Limitando procesamiento a {max_items} de {len(benchmark_data)} preguntas totales")
        data_to_process = benchmark_data[:max_items]
    else:
        data_to_process = benchmark_data
    
    print(f"Filtrando {len(data_to_process)} preguntas...")
    
    for item in tqdm(data_to_process, desc="Filtrando preguntas"):
        pregunta = item.get('pregunta', '')
        respuesta = item.get('respuesta_correcta', '')
        
        # Verificar que tanto pregunta como respuesta estén en español
        if is_spanish_text(pregunta) and is_spanish_text(respuesta):
            filtered_data.append(item)
        else:
            filtered_count += 1
    
    print(f"Filtradas {filtered_count} preguntas (no español)")
    print(f"Preguntas válidas en español: {len(filtered_data)}")
    return filtered_data

def normalizar_texto(texto):
    if not isinstance(texto, str):
        return ""
    texto = texto.lower()
    texto = unidecode(texto)
    texto = texto.translate(str.maketrans('', '', string.punctuation))
    return texto.strip()

# --- Carga benchmark ---
if not os.path.exists(BENCHMARK_FILE):
    print(f"Error: No se encuentra '{BENCHMARK_FILE}'. Ejecuta el script 2 primero.")
    raise SystemExit(1)

with open(BENCHMARK_FILE, 'r', encoding='utf-8') as f:
    benchmark_data = json.load(f)

if len(benchmark_data) == 0:
    print("Error: El benchmark está vacío.")
    raise SystemExit(1)

# Filtrar preguntas en español usando procesamiento secuencial
print(f"Datos originales: {len(benchmark_data)} preguntas")
# benchmark_data_spanish = filter_spanish_questions_sequential(benchmark_data, MAX_PREGUNTAS_FILTRAR)
benchmark_data_spanish = benchmark_data
print(f"Después del filtro de español: {len(benchmark_data_spanish)} preguntas")

if len(benchmark_data_spanish) == 0:
    print("Error: No hay preguntas válidas en español después del filtro.")
    raise SystemExit(1)

# Verificar si tenemos suficientes preguntas para la muestra
if len(benchmark_data_spanish) < CANTIDAD:
    print(f"Advertencia: Solo hay {len(benchmark_data_spanish)} preguntas válidas, usando todas para la evaluación.")
    CANTIDAD = len(benchmark_data_spanish)

benchmark_sample = random.sample(benchmark_data_spanish, min(len(benchmark_data_spanish), CANTIDAD))
print(f"Muestra seleccionada: {len(benchmark_sample)} preguntas")

# --- Modelo local (descarga si hace falta) ---
modelo_local = ensure_local_model(MODELO, MODELS_DIR)
print(f"Modelo preparado en: {modelo_local}")

# --- Tokenizer (para chat template si existe) ---
# Bypass para tokenizer de Gemma
if "google/gemma" in MODELO.lower() and modelo_local == MODELO:
    # Si no se pudo descargar localmente, usar directamente el repo
    print("Cargando tokenizer directamente desde Hugging Face...")
    tok = AutoTokenizer.from_pretrained(
        MODELO,
        trust_remote_code=True
    )
else:
    tok = AutoTokenizer.from_pretrained(
        modelo_local,
        trust_remote_code=True,
        local_files_only=True
    )

def create_few_shot_prompt(pregunta: str) -> str:
    """Crea un prompt con ejemplos few-shot de Latinoamérica"""
    system_message = """Eres un asistente experto en conocimientos de Latinoamérica. Responde de manera precisa y concisa en español.

Ejemplos de preguntas y respuestas sobre Latinoamérica:"""
    
    # Agregar ejemplos few-shot
    examples_text = ""
    for example in FEW_SHOT_EXAMPLES:
        examples_text += f"\nPregunta: {example['pregunta']}\nRespuesta: {example['respuesta']}\n"
    
    user_message = f"\nAhora responde esta pregunta:\nPregunta: {pregunta}\nRespuesta:"
    
    return system_message + examples_text + user_message

def format_prompt(pregunta: str) -> str:
    few_shot_content = create_few_shot_prompt(pregunta)
    
    if hasattr(tok, "apply_chat_template") and (getattr(tok, "chat_template", None) is not None):
        messages = [
            {"role": "system", "content": "Eres un asistente experto en conocimientos de Latinoamérica."},
            {"role": "user", "content": few_shot_content}
        ]
        return tok.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
    return f"<|user|>\n{few_shot_content}<|assistant|>"

prompts = [format_prompt(item['pregunta']) for item in benchmark_sample]
respuestas_correctas_sample = [item['respuesta_correcta'] for item in benchmark_sample]

print(f"\nCargando modelo '{MODELO}' en la GPU...")
sampling_params = SamplingParams(temperature=0.1, top_p=0.9, max_tokens=128)

# Configuración específica para Gemma
if "google/gemma" in MODELO.lower():
    print("Configurando vLLM para modelo Gemma...")
    try:
        llm = LLM(
            model=modelo_local, 
            gpu_memory_utilization=0.7, 
            dtype="float16", 
            trust_remote_code=True,
            # Configuraciones adicionales para Gemma
            max_model_len=4096,
            enforce_eager=True
        )
    except Exception as e:
        print(f"Error con configuración específica de Gemma: {e}")
        print("Intentando con configuración estándar...")
        llm = LLM(model=modelo_local, gpu_memory_utilization=0.7, dtype="float16", trust_remote_code=True)
else:
    llm = LLM(model=modelo_local, gpu_memory_utilization=0.7, dtype="float16", trust_remote_code=True)

print("¡Modelo cargado! Iniciando generación de respuestas...")
outputs = llm.generate(prompts, sampling_params)
print("¡Generación completa! Evaluando respuestas...")

total_preguntas = 0
correctas = 0
resultados_evaluacion = []

for i, output in enumerate(outputs):
    pregunta_original = benchmark_sample[i]['pregunta']
    respuesta_llm = output.outputs[0].text.strip()
    respuesta_correcta_gt = respuestas_correctas_sample[i]

    # Verificar que la respuesta del LLM esté en español
    respuesta_en_espanol = respuesta_llm
    
    norm_correcta = normalizar_texto(respuesta_correcta_gt)
    norm_llm = normalizar_texto(respuesta_llm)
    es_correcta = (norm_correcta in norm_llm) and (norm_correcta != "") and respuesta_en_espanol

    if es_correcta:
        correctas += 1
    total_preguntas += 1

    evaluacion = "CORRECTO" if es_correcta else "INCORRECTO"
    if not respuesta_en_espanol:
        evaluacion += " (No español)"

    resultados_evaluacion.append({
        "pregunta": pregunta_original,
        "respuesta_correcta": respuesta_correcta_gt,
        "respuesta_llm": respuesta_llm,
        "evaluacion": evaluacion,
        "respuesta_en_espanol": respuesta_en_espanol
    })

print("\n--- Tabla de Resultados de Evaluación (Muestra de 10) ---")
for res in resultados_evaluacion[:10]:
    print(f"  P: {res['pregunta']}")
    print(f"  R. Correcta: {res['respuesta_correcta']}")
    print(f"  R. LLM: {res['respuesta_llm']} -> {res['evaluacion']}")
    print("  ---")

if total_preguntas > 0:
    puntaje = (correctas / total_preguntas) * 100
    respuestas_espanol = sum(1 for r in resultados_evaluacion if r['respuesta_en_espanol'])
    print(f"\n Resultado Final ({MODELO}): {correctas} de {total_preguntas} correctas ({puntaje:.1f}%)")
    print(f" Respuestas en español: {respuestas_espanol} de {total_preguntas} ({(respuestas_espanol/total_preguntas)*100:.1f}%)")
else:
    print("No se evaluaron preguntas.")

with open(OUTPUT_FILE, 'w', encoding='utf-8') as f:
    json.dump(resultados_evaluacion, f, ensure_ascii=False, indent=4)

print(f"Resultados detallados guardados en '{OUTPUT_FILE}'")
print("-------------------------------------------------")

In [ ]:
import json
import os
import random
import string
import re
from pathlib import Path
from tqdm import tqdm

from unidecode import unidecode
from vllm import LLM, SamplingParams
from huggingface_hub import snapshot_download
from transformers import AutoTokenizer
from langdetect import detect, detect_langs
from langdetect.lang_detect_exception import LangDetectException
# ------------ CONFIG ------------
MODELO = "Qwen/Qwen3-4B-Instruct-2507"   # repo HF o ruta local meta-llama/Llama-3.1-8B-Instruct TinyLlama/TinyLlama-1.1B-Chat-v1.0 Qwen/Qwen3-4B-Instruct-2507
CANTIDAD = 100000
MAX_PREGUNTAS_FILTRAR = 100000  # Limitar el número de preguntas a procesar para filtrar
BENCHMARK_FILE = "tomy_qa.json"
OUTPUT_FILE = f"tomy_evaluacion_real_{MODELO.split('/')[-1]}.json"
MODELS_DIR = Path("./models")                   # dónde guardar los modelos
os.environ["HUGGINGFACE_HUB_TOKEN"] = ""
# ---------------------------------

print(f"Configuración:")
print(f"- Modelo: {MODELO}")
print(f"- Cantidad de preguntas para evaluación: {CANTIDAD}")
print(f"- Máximo de preguntas a procesar para filtrar: {MAX_PREGUNTAS_FILTRAR}")
print(f"- Archivo benchmark: {BENCHMARK_FILE}")
print("-" * 50)

# Ejemplos few-shot específicos de Latinoamérica
FEW_SHOT_EXAMPLES = [
    {
        "pregunta": "¿Cuál es la capital de Argentina?",
        "respuesta": "Buenos Aires"
    },
    {
        "pregunta": "¿En qué año se independizó México?",
        "respuesta": "1810"
    },
    {
        "pregunta": "¿Cuál es la moneda oficial de Chile?",
        "respuesta": "Peso chileno"
    },
    {
        "pregunta": "¿Quién fue el libertador de Venezuela?",
        "respuesta": "Simón Bolívar"
    },
    {
        "pregunta": "¿Cuál es el río más largo de Sudamérica?",
        "respuesta": "Río Amazonas"
    },
    {
        "pregunta": "¿En qué país se encuentra Machu Picchu?",
        "respuesta": "Perú"
    },
    {
        "pregunta": "¿Cuál es la capital de Colombia?",
        "respuesta": "Bogotá"
    },
    {
        "pregunta": "¿Qué escritor colombiano ganó el Premio Nobel de Literatura?",
        "respuesta": "Gabriel García Márquez"
    },
    {
        "pregunta": "¿Cuál es el país más grande de América del Sur?",
        "respuesta": "Brasil"
    },
    {
        "pregunta": "¿En qué océano se encuentran las Islas Galápagos?",
        "respuesta": "Océano Pacífico"
    },
    {
        "pregunta": "¿Cuál es la capital de Ecuador?",
        "respuesta": "Quito"
    },
    {
        "pregunta": "¿Qué cordillera atraviesa América del Sur?",
        "respuesta": "Cordillera de los Andes"
    },
    {
        "pregunta": "¿Cuál es la moneda oficial de Brasil?",
        "respuesta": "Real brasileño"
    },
    {
        "pregunta": "¿En qué país nació el tango?",
        "respuesta": "Argentina"
    },
    {
        "pregunta": "¿Cuál es la capital de Uruguay?",
        "respuesta": "Montevideo"
    },
    {
        "pregunta": "¿Qué país tiene forma de bota en América del Sur?",
        "respuesta": "Chile"
    },
    {
        "pregunta": "¿Cuál es el desierto más árido del mundo ubicado en Chile?",
        "respuesta": "Desierto de Atacama"
    },
    {
        "pregunta": "¿En qué país se encuentra el Salar de Uyuni?",
        "respuesta": "Bolivia"
    },
    {
        "pregunta": "¿Cuál es la capital de Paraguay?",
        "respuesta": "Asunción"
    },
    {
        "pregunta": "¿Qué idioma se habla en Brasil?",
        "respuesta": "Portugués"
    }
]

def is_local_path(p: str) -> bool:
    return Path(p).expanduser().exists()

def sanitize(name: str) -> str:
    return "".join(ch if ch.isalnum() or ch in "._-+=" else "_" for ch in name)

def ensure_local_model(model_id_or_path: str, local_dir: Path) -> str:
    if is_local_path(model_id_or_path):
        return str(Path(model_id_or_path).expanduser().resolve())
    
    local_dir.mkdir(parents=True, exist_ok=True)
    dst = local_dir / sanitize(model_id_or_path)
    
    # Bypass específico para google/gemma-3-4b-it
    if "google/gemma" in model_id_or_path.lower():
        print(f"Aplicando bypass para modelo Gemma: {model_id_or_path}")
        try:
            # Intentar descarga con patrones más amplios para Gemma
            snapshot_download(
                repo_id=model_id_or_path,
                local_dir=str(dst),
                local_dir_use_symlinks=False,
                # Patrones más amplios para Gemma
                allow_patterns=[
                    "*.json", "*.txt", "*.model", "*.safetensors", "*.bin",
                    "tokenizer*", "config*", "generation_config*",
                    "model*", "pytorch_model*", "special_tokens_map*"
                ],
            )
        except Exception as e1:
            print(f"Primer intento falló: {e1}")
            try:
                # Segundo intento: descarga completa sin filtros
                print("Intentando descarga completa sin filtros...")
                snapshot_download(
                    repo_id=model_id_or_path,
                    local_dir=str(dst),
                    local_dir_use_symlinks=False,
                )
            except Exception as e2:
                print(f"Segundo intento falló: {e2}")
                # Tercer intento: usar directamente el repo_id para vLLM
                print("Usando repo_id directamente para vLLM...")
                return model_id_or_path
    else:
        # Para otros modelos, usar el método original
        snapshot_download(
            repo_id=model_id_or_path,
            local_dir=str(dst),
            local_dir_use_symlinks=False,
            allow_patterns=[
                "config.json", "generation_config.json",
                "tokenizer.json", "tokenizer.model", "tokenizer_config.json",
                "special_tokens_map.json", "merges.txt", "vocab.json",
                "*.safetensors", "model.safetensors.index.json", "pytorch_model.bin",
            ],
        )
    
    return str(dst)

def is_spanish_text(text: str) -> bool:
    """Detecta si el texto está en español"""
    if not text or len(text.strip()) < 3:
        return False
    
    try:
        # Usar detect_langs para obtener probabilidades
        languages = detect_langs(text)
        
        # Buscar español en los resultados
        for lang in languages:
            if lang.lang == 'es' and lang.prob > 0.55:
                return True
        
        # Si no hay alta confianza, usar detect simple
        detected_lang = detect(text)
        return detected_lang == 'es'
        
    except:
        # Si langdetect falla completamente, considerar como no español
        return False

def filter_spanish_questions_sequential(benchmark_data, max_items=None):
    """Filtra preguntas que estén completamente en español usando procesamiento secuencial"""
    filtered_data = []
    filtered_count = 0
    
    # Limitar el número de items a procesar si se especifica
    if max_items and max_items < len(benchmark_data):
        print(f"Limitando procesamiento a {max_items} de {len(benchmark_data)} preguntas totales")
        data_to_process = benchmark_data[:max_items]
    else:
        data_to_process = benchmark_data
    
    print(f"Filtrando {len(data_to_process)} preguntas...")
    
    for item in tqdm(data_to_process, desc="Filtrando preguntas"):
        pregunta = item.get('pregunta', '')
        respuesta = item.get('respuesta_correcta', '')
        
        # Verificar que tanto pregunta como respuesta estén en español
        if is_spanish_text(pregunta) and is_spanish_text(respuesta):
            filtered_data.append(item)
        else:
            filtered_count += 1
    
    print(f"Filtradas {filtered_count} preguntas (no español)")
    print(f"Preguntas válidas en español: {len(filtered_data)}")
    return filtered_data

def normalizar_texto(texto):
    if not isinstance(texto, str):
        return ""
    texto = texto.lower()
    texto = unidecode(texto)
    texto = texto.translate(str.maketrans('', '', string.punctuation))
    return texto.strip()

# --- Carga benchmark ---
if not os.path.exists(BENCHMARK_FILE):
    print(f"Error: No se encuentra '{BENCHMARK_FILE}'. Ejecuta el script 2 primero.")
    raise SystemExit(1)

with open(BENCHMARK_FILE, 'r', encoding='utf-8') as f:
    benchmark_data = json.load(f)

if len(benchmark_data) == 0:
    print("Error: El benchmark está vacío.")
    raise SystemExit(1)

# Filtrar preguntas en español usando procesamiento secuencial
print(f"Datos originales: {len(benchmark_data)} preguntas")
# benchmark_data_spanish = filter_spanish_questions_sequential(benchmark_data, MAX_PREGUNTAS_FILTRAR)
benchmark_data_spanish = benchmark_data
print(f"Después del filtro de español: {len(benchmark_data_spanish)} preguntas")

if len(benchmark_data_spanish) == 0:
    print("Error: No hay preguntas válidas en español después del filtro.")
    raise SystemExit(1)

# Verificar si tenemos suficientes preguntas para la muestra
if len(benchmark_data_spanish) < CANTIDAD:
    print(f"Advertencia: Solo hay {len(benchmark_data_spanish)} preguntas válidas, usando todas para la evaluación.")
    CANTIDAD = len(benchmark_data_spanish)

benchmark_sample = random.sample(benchmark_data_spanish, min(len(benchmark_data_spanish), CANTIDAD))
print(f"Muestra seleccionada: {len(benchmark_sample)} preguntas")

# --- Modelo local (descarga si hace falta) ---
modelo_local = ensure_local_model(MODELO, MODELS_DIR)
print(f"Modelo preparado en: {modelo_local}")

# --- Tokenizer (para chat template si existe) ---
# Bypass para tokenizer de Gemma
if "google/gemma" in MODELO.lower() and modelo_local == MODELO:
    # Si no se pudo descargar localmente, usar directamente el repo
    print("Cargando tokenizer directamente desde Hugging Face...")
    tok = AutoTokenizer.from_pretrained(
        MODELO,
        trust_remote_code=True
    )
else:
    tok = AutoTokenizer.from_pretrained(
        modelo_local,
        trust_remote_code=True,
        local_files_only=True
    )

def create_few_shot_prompt(pregunta: str) -> str:
    """Crea un prompt con ejemplos few-shot de Latinoamérica"""
    system_message = """Eres un asistente experto en conocimientos de Latinoamérica. Responde de manera precisa y concisa en español.

Ejemplos de preguntas y respuestas sobre Latinoamérica:"""
    
    # Agregar ejemplos few-shot
    examples_text = ""
    for example in FEW_SHOT_EXAMPLES:
        examples_text += f"\nPregunta: {example['pregunta']}\nRespuesta: {example['respuesta']}\n"
    
    user_message = f"\nAhora responde esta pregunta:\nPregunta: {pregunta}\nRespuesta:"
    
    return system_message + examples_text + user_message

def format_prompt(pregunta: str) -> str:
    few_shot_content = create_few_shot_prompt(pregunta)
    
    if hasattr(tok, "apply_chat_template") and (getattr(tok, "chat_template", None) is not None):
        messages = [
            {"role": "system", "content": "Eres un asistente experto en conocimientos de Latinoamérica."},
            {"role": "user", "content": few_shot_content}
        ]
        return tok.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
    return f"<|user|>\n{few_shot_content}<|assistant|>"

prompts = [format_prompt(item['pregunta']) for item in benchmark_sample]
respuestas_correctas_sample = [item['respuesta_correcta'] for item in benchmark_sample]

print(f"\nCargando modelo '{MODELO}' en la GPU...")
sampling_params = SamplingParams(temperature=0.1, top_p=0.9, max_tokens=128)

# Configuración específica para Gemma
if "google/gemma" in MODELO.lower():
    print("Configurando vLLM para modelo Gemma...")
    try:
        llm = LLM(
            model=modelo_local, 
            gpu_memory_utilization=0.7, 
            dtype="float16", 
            trust_remote_code=True,
            # Configuraciones adicionales para Gemma
            max_model_len=4096,
            enforce_eager=True
        )
    except Exception as e:
        print(f"Error con configuración específica de Gemma: {e}")
        print("Intentando con configuración estándar...")
        llm = LLM(model=modelo_local, gpu_memory_utilization=0.7, dtype="float16", trust_remote_code=True)
else:
    llm = LLM(model=modelo_local, gpu_memory_utilization=0.7, dtype="float16", trust_remote_code=True)

print("¡Modelo cargado! Iniciando generación de respuestas...")
outputs = llm.generate(prompts, sampling_params)
print("¡Generación completa! Evaluando respuestas...")

total_preguntas = 0
correctas = 0
resultados_evaluacion = []

for i, output in enumerate(outputs):
    pregunta_original = benchmark_sample[i]['pregunta']
    respuesta_llm = output.outputs[0].text.strip()
    respuesta_correcta_gt = respuestas_correctas_sample[i]

    # Verificar que la respuesta del LLM esté en español
    respuesta_en_espanol = respuesta_llm
    
    norm_correcta = normalizar_texto(respuesta_correcta_gt)
    norm_llm = normalizar_texto(respuesta_llm)
    es_correcta = (norm_correcta in norm_llm) and (norm_correcta != "") and respuesta_en_espanol

    if es_correcta:
        correctas += 1
    total_preguntas += 1

    evaluacion = "CORRECTO" if es_correcta else "INCORRECTO"
    if not respuesta_en_espanol:
        evaluacion += " (No español)"

    resultados_evaluacion.append({
        "pregunta": pregunta_original,
        "respuesta_correcta": respuesta_correcta_gt,
        "respuesta_llm": respuesta_llm,
        "evaluacion": evaluacion,
        "respuesta_en_espanol": respuesta_en_espanol
    })

print("\n--- Tabla de Resultados de Evaluación (Muestra de 10) ---")
for res in resultados_evaluacion[:10]:
    print(f"  P: {res['pregunta']}")
    print(f"  R. Correcta: {res['respuesta_correcta']}")
    print(f"  R. LLM: {res['respuesta_llm']} -> {res['evaluacion']}")
    print("  ---")

if total_preguntas > 0:
    puntaje = (correctas / total_preguntas) * 100
    respuestas_espanol = sum(1 for r in resultados_evaluacion if r['respuesta_en_espanol'])
    print(f"\n Resultado Final ({MODELO}): {correctas} de {total_preguntas} correctas ({puntaje:.1f}%)")
    print(f" Respuestas en español: {respuestas_espanol} de {total_preguntas} ({(respuestas_espanol/total_preguntas)*100:.1f}%)")
else:
    print("No se evaluaron preguntas.")

with open(OUTPUT_FILE, 'w', encoding='utf-8') as f:
    json.dump(resultados_evaluacion, f, ensure_ascii=False, indent=4)

print(f"Resultados detallados guardados en '{OUTPUT_FILE}'")
print("-------------------------------------------------")

In [ ]:
import json
import os
import random
import string
import re
from pathlib import Path
from tqdm import tqdm

from unidecode import unidecode
from vllm import LLM, SamplingParams
from huggingface_hub import snapshot_download
from transformers import AutoTokenizer
from langdetect import detect, detect_langs
from langdetect.lang_detect_exception import LangDetectException

# ------------ CONFIG ------------
MODELO = "Qwen/Qwen3-4B-Instruct-2507"   # repo HF o ruta local meta-llama/Llama-3.1-8B-Instruct TinyLlama/TinyLlama-1.1B-Chat-v1.0 Qwen/Qwen3-4B-Instruct-2507
CANTIDAD = 500000
MAX_PREGUNTAS_FILTRAR = 500000  # Limitar el número de preguntas a procesar para filtrar
BENCHMARK_FILE = "justo_qa.json"
OUTPUT_FILE = f"justo_evaluacion_real_{MODELO.split('/')[-1]}.json"
MODELS_DIR = Path("./models")                   # dónde guardar los modelos
os.environ["HUGGINGFACE_HUB_TOKEN"] = ""
# ---------------------------------

print(f"Configuración:")
print(f"- Modelo: {MODELO}")
print(f"- Cantidad de preguntas para evaluación: {CANTIDAD}")
print(f"- Máximo de preguntas a procesar para filtrar: {MAX_PREGUNTAS_FILTRAR}")
print(f"- Archivo benchmark: {BENCHMARK_FILE}")
print("-" * 50)

# Ejemplos few-shot específicos de Latinoamérica
FEW_SHOT_EXAMPLES = [
    {
        "pregunta": "¿Cuál es la capital de Argentina?",
        "respuesta": "Buenos Aires"
    },
    {
        "pregunta": "¿En qué año se independizó México?",
        "respuesta": "1810"
    },
    {
        "pregunta": "¿Cuál es la moneda oficial de Chile?",
        "respuesta": "Peso chileno"
    },
    {
        "pregunta": "¿Quién fue el libertador de Venezuela?",
        "respuesta": "Simón Bolívar"
    },
    {
        "pregunta": "¿Cuál es el río más largo de Sudamérica?",
        "respuesta": "Río Amazonas"
    },
    {
        "pregunta": "¿En qué país se encuentra Machu Picchu?",
        "respuesta": "Perú"
    },
    {
        "pregunta": "¿Cuál es la capital de Colombia?",
        "respuesta": "Bogotá"
    },
    {
        "pregunta": "¿Qué escritor colombiano ganó el Premio Nobel de Literatura?",
        "respuesta": "Gabriel García Márquez"
    },
    {
        "pregunta": "¿Cuál es el país más grande de América del Sur?",
        "respuesta": "Brasil"
    },
    {
        "pregunta": "¿En qué océano se encuentran las Islas Galápagos?",
        "respuesta": "Océano Pacífico"
    },
    {
        "pregunta": "¿Cuál es la capital de Ecuador?",
        "respuesta": "Quito"
    },
    {
        "pregunta": "¿Qué cordillera atraviesa América del Sur?",
        "respuesta": "Cordillera de los Andes"
    },
    {
        "pregunta": "¿Cuál es la moneda oficial de Brasil?",
        "respuesta": "Real brasileño"
    },
    {
        "pregunta": "¿En qué país nació el tango?",
        "respuesta": "Argentina"
    },
    {
        "pregunta": "¿Cuál es la capital de Uruguay?",
        "respuesta": "Montevideo"
    },
    {
        "pregunta": "¿Qué país tiene forma de bota en América del Sur?",
        "respuesta": "Chile"
    },
    {
        "pregunta": "¿Cuál es el desierto más árido del mundo ubicado en Chile?",
        "respuesta": "Desierto de Atacama"
    },
    {
        "pregunta": "¿En qué país se encuentra el Salar de Uyuni?",
        "respuesta": "Bolivia"
    },
    {
        "pregunta": "¿Cuál es la capital de Paraguay?",
        "respuesta": "Asunción"
    },
    {
        "pregunta": "¿Qué idioma se habla en Brasil?",
        "respuesta": "Portugués"
    }
]

def is_local_path(p: str) -> bool:
    return Path(p).expanduser().exists()

def sanitize(name: str) -> str:
    return "".join(ch if ch.isalnum() or ch in "._-+=" else "_" for ch in name)

def ensure_local_model(model_id_or_path: str, local_dir: Path) -> str:
    if is_local_path(model_id_or_path):
        return str(Path(model_id_or_path).expanduser().resolve())
    
    local_dir.mkdir(parents=True, exist_ok=True)
    dst = local_dir / sanitize(model_id_or_path)
    
    # Bypass específico para google/gemma-3-4b-it
    if "google/gemma" in model_id_or_path.lower():
        print(f"Aplicando bypass para modelo Gemma: {model_id_or_path}")
        try:
            # Intentar descarga con patrones más amplios para Gemma
            snapshot_download(
                repo_id=model_id_or_path,
                local_dir=str(dst),
                local_dir_use_symlinks=False,
                # Patrones más amplios para Gemma
                allow_patterns=[
                    "*.json", "*.txt", "*.model", "*.safetensors", "*.bin",
                    "tokenizer*", "config*", "generation_config*",
                    "model*", "pytorch_model*", "special_tokens_map*"
                ],
            )
        except Exception as e1:
            print(f"Primer intento falló: {e1}")
            try:
                # Segundo intento: descarga completa sin filtros
                print("Intentando descarga completa sin filtros...")
                snapshot_download(
                    repo_id=model_id_or_path,
                    local_dir=str(dst),
                    local_dir_use_symlinks=False,
                )
            except Exception as e2:
                print(f"Segundo intento falló: {e2}")
                # Tercer intento: usar directamente el repo_id para vLLM
                print("Usando repo_id directamente para vLLM...")
                return model_id_or_path
    else:
        # Para otros modelos, usar el método original
        snapshot_download(
            repo_id=model_id_or_path,
            local_dir=str(dst),
            local_dir_use_symlinks=False,
            allow_patterns=[
                "config.json", "generation_config.json",
                "tokenizer.json", "tokenizer.model", "tokenizer_config.json",
                "special_tokens_map.json", "merges.txt", "vocab.json",
                "*.safetensors", "model.safetensors.index.json", "pytorch_model.bin",
            ],
        )
    
    return str(dst)

def is_spanish_text(text: str) -> bool:
    """Detecta si el texto está en español"""
    if not text or len(text.strip()) < 3:
        return False
    
    try:
        # Usar detect_langs para obtener probabilidades
        languages = detect_langs(text)
        
        # Buscar español en los resultados
        for lang in languages:
            if lang.lang == 'es' and lang.prob > 0.55:
                return True
        
        # Si no hay alta confianza, usar detect simple
        detected_lang = detect(text)
        return detected_lang == 'es'
        
    except:
        # Si langdetect falla completamente, considerar como no español
        return False

def filter_spanish_questions_sequential(benchmark_data, max_items=None):
    """Filtra preguntas que estén completamente en español usando procesamiento secuencial"""
    filtered_data = []
    filtered_count = 0
    
    # Limitar el número de items a procesar si se especifica
    if max_items and max_items < len(benchmark_data):
        print(f"Limitando procesamiento a {max_items} de {len(benchmark_data)} preguntas totales")
        data_to_process = benchmark_data[:max_items]
    else:
        data_to_process = benchmark_data
    
    print(f"Filtrando {len(data_to_process)} preguntas...")
    
    for item in tqdm(data_to_process, desc="Filtrando preguntas"):
        pregunta = item.get('pregunta', '')
        respuesta = item.get('respuesta_correcta', '')
        
        # Verificar que tanto pregunta como respuesta estén en español
        if is_spanish_text(pregunta) and is_spanish_text(respuesta):
            filtered_data.append(item)
        else:
            filtered_count += 1
    
    print(f"Filtradas {filtered_count} preguntas (no español)")
    print(f"Preguntas válidas en español: {len(filtered_data)}")
    return filtered_data

def normalizar_texto(texto):
    if not isinstance(texto, str):
        return ""
    texto = texto.lower()
    texto = unidecode(texto)
    texto = texto.translate(str.maketrans('', '', string.punctuation))
    return texto.strip()

# --- Carga benchmark ---
if not os.path.exists(BENCHMARK_FILE):
    print(f"Error: No se encuentra '{BENCHMARK_FILE}'. Ejecuta el script 2 primero.")
    raise SystemExit(1)

with open(BENCHMARK_FILE, 'r', encoding='utf-8') as f:
    benchmark_data = json.load(f)

if len(benchmark_data) == 0:
    print("Error: El benchmark está vacío.")
    raise SystemExit(1)

# Filtrar preguntas en español usando procesamiento secuencial
print(f"Datos originales: {len(benchmark_data)} preguntas")
# benchmark_data_spanish = filter_spanish_questions_sequential(benchmark_data, MAX_PREGUNTAS_FILTRAR)
benchmark_data_spanish = benchmark_data
print(f"Después del filtro de español: {len(benchmark_data_spanish)} preguntas")

if len(benchmark_data_spanish) == 0:
    print("Error: No hay preguntas válidas en español después del filtro.")
    raise SystemExit(1)

# Verificar si tenemos suficientes preguntas para la muestra
if len(benchmark_data_spanish) < CANTIDAD:
    print(f"Advertencia: Solo hay {len(benchmark_data_spanish)} preguntas válidas, usando todas para la evaluación.")
    CANTIDAD = len(benchmark_data_spanish)

benchmark_sample = random.sample(benchmark_data_spanish, min(len(benchmark_data_spanish), CANTIDAD))
print(f"Muestra seleccionada: {len(benchmark_sample)} preguntas")

# --- Modelo local (descarga si hace falta) ---
modelo_local = ensure_local_model(MODELO, MODELS_DIR)
print(f"Modelo preparado en: {modelo_local}")

# --- Tokenizer (para chat template si existe) ---
# Bypass para tokenizer de Gemma
if "google/gemma" in MODELO.lower() and modelo_local == MODELO:
    # Si no se pudo descargar localmente, usar directamente el repo
    print("Cargando tokenizer directamente desde Hugging Face...")
    tok = AutoTokenizer.from_pretrained(
        MODELO,
        trust_remote_code=True
    )
else:
    tok = AutoTokenizer.from_pretrained(
        modelo_local,
        trust_remote_code=True,
        local_files_only=True
    )

def create_few_shot_prompt(pregunta: str) -> str:
    """Crea un prompt con ejemplos few-shot de Latinoamérica"""
    system_message = """Eres un asistente experto en conocimientos de Latinoamérica. Responde de manera precisa y concisa en español.

Ejemplos de preguntas y respuestas sobre Latinoamérica:"""
    
    # Agregar ejemplos few-shot
    examples_text = ""
    for example in FEW_SHOT_EXAMPLES:
        examples_text += f"\nPregunta: {example['pregunta']}\nRespuesta: {example['respuesta']}\n"
    
    user_message = f"\nAhora responde esta pregunta:\nPregunta: {pregunta}\nRespuesta:"
    
    return system_message + examples_text + user_message

def format_prompt(pregunta: str) -> str:
    few_shot_content = create_few_shot_prompt(pregunta)
    
    if hasattr(tok, "apply_chat_template") and (getattr(tok, "chat_template", None) is not None):
        messages = [
            {"role": "system", "content": "Eres un asistente experto en conocimientos de Latinoamérica."},
            {"role": "user", "content": few_shot_content}
        ]
        return tok.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
    return f"<|user|>\n{few_shot_content}<|assistant|>"

prompts = [format_prompt(item['pregunta']) for item in benchmark_sample]
respuestas_correctas_sample = [item['respuesta_correcta'] for item in benchmark_sample]

print(f"\nCargando modelo '{MODELO}' en la GPU...")
sampling_params = SamplingParams(temperature=0.1, top_p=0.9, max_tokens=128)

# Configuración específica para Gemma
if "google/gemma" in MODELO.lower():
    print("Configurando vLLM para modelo Gemma...")
    try:
        llm = LLM(
            model=modelo_local, 
            gpu_memory_utilization=0.9, 
            dtype="float16", 
            trust_remote_code=True,
            # Configuraciones adicionales para Gemma
            max_model_len=4096,
            enforce_eager=True
        )
    except Exception as e:
        print(f"Error con configuración específica de Gemma: {e}")
        print("Intentando con configuración estándar...")
        llm = LLM(model=modelo_local, gpu_memory_utilization=0.9, dtype="float16", trust_remote_code=True)
else:
    llm = LLM(model=modelo_local, gpu_memory_utilization=0.9, dtype="float16", trust_remote_code=True, tensor_parallel_size=4)

print("¡Modelo cargado! Iniciando generación de respuestas...")
outputs = llm.generate(prompts, sampling_params)
print("¡Generación completa! Evaluando respuestas...")

total_preguntas = 0
correctas = 0
resultados_evaluacion = []

for i, output in enumerate(outputs):
    pregunta_original = benchmark_sample[i]['pregunta']
    respuesta_llm = output.outputs[0].text.strip()
    respuesta_correcta_gt = respuestas_correctas_sample[i]

    # Verificar que la respuesta del LLM esté en español
    respuesta_en_espanol = respuesta_llm
    
    norm_correcta = normalizar_texto(respuesta_correcta_gt)
    norm_llm = normalizar_texto(respuesta_llm)
    es_correcta = (norm_correcta in norm_llm) and (norm_correcta != "") and respuesta_en_espanol

    if es_correcta:
        correctas += 1
    total_preguntas += 1

    evaluacion = "CORRECTO" if es_correcta else "INCORRECTO"
    if not respuesta_en_espanol:
        evaluacion += " (No español)"

    resultados_evaluacion.append({
        "pregunta": pregunta_original,
        "respuesta_correcta": respuesta_correcta_gt,
        "respuesta_llm": respuesta_llm,
        "evaluacion": evaluacion,
        "respuesta_en_espanol": respuesta_en_espanol
    })

print("\n--- Tabla de Resultados de Evaluación (Muestra de 10) ---")
for res in resultados_evaluacion[:10]:
    print(f"  P: {res['pregunta']}")
    print(f"  R. Correcta: {res['respuesta_correcta']}")
    print(f"  R. LLM: {res['respuesta_llm']} -> {res['evaluacion']}")
    print("  ---")

if total_preguntas > 0:
    puntaje = (correctas / total_preguntas) * 100
    respuestas_espanol = sum(1 for r in resultados_evaluacion if r['respuesta_en_espanol'])
    print(f"\n Resultado Final ({MODELO}): {correctas} de {total_preguntas} correctas ({puntaje:.1f}%)")
    print(f" Respuestas en español: {respuestas_espanol} de {total_preguntas} ({(respuestas_espanol/total_preguntas)*100:.1f}%)")
else:
    print("No se evaluaron preguntas.")

with open(OUTPUT_FILE, 'w', encoding='utf-8') as f:
    json.dump(resultados_evaluacion, f, ensure_ascii=False, indent=4)

print(f"Resultados detallados guardados en '{OUTPUT_FILE}'")
print("-------------------------------------------------")